# Groove MIDI Dataset을 이용한 MusicVAE 기반 4마디 드럼 샘플 만들기

## 1. Magenta github clone & Setup

In [ ]:
!git clone https://github.com/tensorflow/magenta.git
!sudo apt install -y fluidsynth
!pip install pretty_midi

Magenta github를 clone 해와서 만들어진 파이프라인을 바탕으로 학습을 하고 기본적인 셋팅을 실행합니다.

In [3]:
cd magenta

/content/magenta


clone으로 가져온 magenta 폴더로 이동합니다.

In [ ]:
!pip install -e .

magenta 패키지에 종속되는 library들을 설치해줍니다.

In [5]:
import os
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
os.chdir("/content/magenta/magenta/models/music_vae/")

DeprecationWarning 이 나오는 warning은 무시하고 music_vae를 학습하기 위해서 music_vae_train.py 파일이 있는 경로로 이동합니다.

## 2. Training

In [ ]:
!python music_vae_train.py \
 --config=groovae_4bar \
 --run_dir=/tmp/groovae_4bar \
 --mode=train \
 --tfds_name=groove/4bar-midionly \
 --num_steps=100

[music_vae_train](https://github.com/magenta/magenta/blob/main/magenta/models/music_vae/music_vae_train.py)
- config : 사전 학습에 정의된 groovae_4bar라는 이름의 config를 설정합니다.
  - 설정 가능한 config list 는 해당 [링크](https://github.com/magenta/magenta/tree/main/magenta/models/music_vae)에 있습니다.
  - groovae_4bar의 의미는 4-bar로 분할된 groove 데이터셋을 사용한 autoencoder인 config를 의미합니다.
- run_dir : 학습을 시작할 작업환경 디렉토리를 설정합니다.
- mode : train을 하면 학습모드이고 default이면 평가모드입니다.
- tfds_name : 텐서플로우에서 제공하는 4-bar로 분할된 groove midionly 데이터셋을 불러와서 학습에 사용합니다.
  - [Tensorflow groove dataset list](https://www.tensorflow.org/datasets/catalog/groove)
- num_steps : 학습하는데 실행되는 step수로 None이면 무한정 진행합니다.

## 3. Generating

In [ ]:
!python music_vae_generate.py \
 --config=groovae_4bar \
 --checkpoint_file=/tmp/groovae_4bar/train/model.ckpt-53 \
 --mode=sample \
 --num_outputs=5 \
 --output_dir=/tmp/generated

[music_vae_generate](https://github.com/magenta/magenta/blob/main/magenta/models/music_vae/music_vae_generate.py)
- config : 사전 학습에 정의된 config를 설정하여 불러옵니다.
- checkpoint_file : training에서 학습하는 동안 저장된 모델의 체크포인트를 불러옵니다.
- mode : 학습된 모델로 sample을 만들거나 시퀀스사이를 interpolate합니다.
- num_outputs : sample로 하는 경우 생성한 sample의 개수를 말합니다.
- output_dir : 생성된 midi파일을 저장할 디렉토리 위치를 정합니다.

## 4. Testing

In [1]:
import numpy as np
import pretty_midi
from glob import glob
import tensorflow as tf
from IPython import display

필요한 패키지들을 불러와줍니다.

In [2]:
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=30):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # midi파일의 30초까지 sequence를 불러옵니다.
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)

midi 파일을 들을 수 있는 오디오 플레이어로 표현해주는 함수입니다.

In [5]:
i = 0
example_path = glob("/tmp/generated/*")
for path in example_path:
  print(path)
pm = pretty_midi.PrettyMIDI(example_path[i])

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Sampling rate for audio playback
_SAMPLING_RATE = 16000

display_audio(pm)

/tmp/generated/groovae_4bar_sample_2022-05-03_171726-000-of-005.mid
/tmp/generated/groovae_4bar_sample_2022-05-03_171726-001-of-005.mid
/tmp/generated/groovae_4bar_sample_2022-05-03_171726-003-of-005.mid
/tmp/generated/groovae_4bar_sample_2022-05-03_171726-004-of-005.mid
/tmp/generated/groovae_4bar_sample_2022-05-03_171726-002-of-005.mid
